In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import mlflow

In [0]:
project = dataiku.api_client().get_default_project()
managed_folder = project.get_managed_folder('A_MANAGED_FOLDER_ID')

with project.setup_mlflow(managed_folder=managed_folder) as mlflow:

    # Note: if you don't call this (i.e. when no experiment is specified), the default one is used
    mlflow.set_experiment("My first experiment")

    with mlflow.start_run(run_name="my_run"):
        # ...your MLflow code...
        mlflow.log_param("a", 1)
        mlflow.log_metric("b", 2)

        # This uses the regular MLflow APIs

In [0]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("mydataset")
mydataset_df = mydataset.get_dataframe()